# Yelp Data Cleaning: Noise Reduction (Chunking Refactor)

## Phase 0: Setup and Initial Load

In [ ]:
import pandas as pd
import numpy as np
import json
import datetime
import re
import os
import random
import gc # Garbage Collector
from sklearn.ensemble import IsolationForest
import nltk # Uncomment if using stopwords in Phase 2
from nltk.corpus import stopwords # Uncomment if using stopwords
from nltk.tokenize import word_tokenize # Uncomment if using stopwords
# Potential plotting libraries (optional)
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
pd.set_option('display.max_columns', None) # Show all columns

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wiztu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# Define File Paths
input_dir = '../yelp_dataset/' # Relative to script location
review_file = os.path.join(input_dir, 'yelp_academic_dataset_review.json')
user_file = os.path.join(input_dir, 'yelp_academic_dataset_user.json')
business_file = os.path.join(input_dir, 'yelp_academic_dataset_business.json')
# tip_file = os.path.join(input_dir, 'yelp_academic_dataset_tip.json') # Optional

# Define Output Directory
output_dir = os.path.join(input_dir, 'cleaned_data')
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory created/exists: {output_dir}")

Output directory created/exists: ../yelp_dataset/cleaned_data


In [3]:
# Load Static Data (Users and Business)
print("Loading user data...")
users_df = pd.read_json(user_file, lines=True)
print("Loading business data...")
business_df = pd.read_json(business_file, lines=True)
print("Static data loading complete.")

print("--- Initial Static Data Shapes ---")
print(f"Users: {users_df.shape}")     # Expected: (1987897, 22)
print(f"Business: {business_df.shape}") # Expected: (150346, 14)

print("\n--- User Columns & Head ---")
print(users_df.columns)
print(users_df.head(2))

print("\n--- Business Columns & Head ---")
print(business_df.columns)
print(business_df.head(2))

print("\n--- Missing Values (Static) ---")
print("\nUsers:\n", users_df.isnull().sum())
print("\nBusiness:\n", business_df.isnull().sum()) # Address/postal code NaNs expected

Loading user data...
Loading business data...
Static data loading complete.
--- Initial Static Data Shapes ---
Users: (1987897, 22)
Business: (150346, 14)

--- User Columns & Head ---
Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')
                  user_id    name  review_count        yelping_since  useful  \
0  qVc8ODYU5SZjKXVBgXdI7w  Walker           585  2007-01-25 16:47:26    7217   
1  j14WgRoU_-2ZE1aw1dXrJg  Daniel          4333  2009-01-25 04:35:42   43091   

   funny   cool                                              elite  \
0   1259   5994                                               2007   
1  13066  27281  2009,2010,20

In [11]:
# Define helper function (moved from original Phase 1)
def count_friends(friends_str):
    if friends_str is None or friends_str == 'None' or pd.isna(friends_str):
        return 0
    return len([f for f in friends_str.split(',') if f])

# Define URL pattern (moved from original Phase 2)
url_pattern = r'http[s]?://|www\\.|\\S+\\.(com|net|org|edu|gov)\\S*|\\S+@\\S+'

## Chunk Processing Loop (Reviews)

In [5]:
chunk_size = 500000
processed_chunks = []
total_reviews_processed = 0
total_reviews_kept_chunk_phase = 0

print(f"Starting review processing in chunks of {chunk_size}...")

for i, review_chunk in enumerate(pd.read_json(review_file, lines=True, chunksize=chunk_size)):
    print(f"\n--- Processing Chunk {i+1} --- ({len(review_chunk)} reviews)")
    total_reviews_processed += len(review_chunk)

    # 1. Merge with Users
    print("  Merging with users...")
    chunk_merged = pd.merge(review_chunk, users_df, on='user_id', how='inner', suffixes=('_review', '_user'))
    print(f"  Reviews after merge: {len(chunk_merged)} (Removed {len(review_chunk) - len(chunk_merged)} due to missing users)")
    del review_chunk # Free memory
    gc.collect()

    # 2. Feature Engineering (Chunk-level)
    print("  Performing feature engineering...")
    chunk_merged['date'] = pd.to_datetime(chunk_merged['date'], errors='coerce')
    chunk_merged['yelping_since'] = pd.to_datetime(chunk_merged['yelping_since'], errors='coerce')
    # Drop rows where date conversion failed
    initial_chunk_count = len(chunk_merged)
    chunk_merged.dropna(subset=['date', 'yelping_since'], inplace=True)
    print(f"    Dropped {initial_chunk_count - len(chunk_merged)} rows due to NaT dates.")

    chunk_merged['text_length'] = chunk_merged['text'].astype(str).str.len()
    compliment_cols = [col for col in chunk_merged.columns if col.startswith('compliment_')]
    chunk_merged['user_compliments_total'] = chunk_merged[compliment_cols].sum(axis=1)
    chunk_merged['user_friends_count'] = chunk_merged['friends'].apply(count_friends)
    if 'elite' in chunk_merged.columns:
        chunk_merged['user_is_elite'] = chunk_merged['elite'].apply(lambda x: isinstance(x, str) and x != '' and x is not None)
    else:
        chunk_merged['user_is_elite'] = False

    # 3. Apply Review-Level Filters (A, F, 2A)
    print("  Applying review-level filters...")
    count_before_filters = len(chunk_merged)

    # Filter A: Text Length
    MIN_REVIEW_LENGTH = 50
    chunk_merged = chunk_merged[chunk_merged['text_length'] >= MIN_REVIEW_LENGTH]
    print(f"    Filter A (Text Length >= {MIN_REVIEW_LENGTH}): Kept {len(chunk_merged)} reviews")

    # Filter F: Low Usefulness (Old/Short/0-useful)
    MAX_REVIEW_AGE_YEARS = 2
    cutoff_date = pd.Timestamp.now(tz='UTC') - pd.DateOffset(years=MAX_REVIEW_AGE_YEARS)
    if chunk_merged['date'].dt.tz is not None:
        cutoff_date = cutoff_date.tz_convert(chunk_merged['date'].dt.tz)
    else:
        cutoff_date = cutoff_date.tz_localize(None)
    condition_f = (
        (chunk_merged['date'] < cutoff_date) & 
        (chunk_merged['useful_review'] == 0) & 
        (chunk_merged['text_length'] < 150)
    )
    chunk_merged = chunk_merged[~condition_f]
    print(f"    Filter F (Low Usefulness): Kept {len(chunk_merged)} reviews")

    # Filter 2A: URLs/Emails
    contains_url_mask = chunk_merged['text'].str.contains(url_pattern, regex=True, na=False)
    chunk_merged = chunk_merged[~contains_url_mask]
    print(f"    Filter 2A (URLs/Emails): Kept {len(chunk_merged)} reviews")

    reviews_removed_in_chunk = count_before_filters - len(chunk_merged)
    print(f"  Total reviews removed in this chunk by review-level filters: {reviews_removed_in_chunk}")
    total_reviews_kept_chunk_phase += len(chunk_merged)

    # 4. Append to list (select necessary columns to save memory)
    # Keep all columns needed for Phase 1 filters + final output
    cols_to_keep_for_phase1 = [
        'review_id', 'user_id', 'business_id', 'stars', 
        'useful_review', 'funny_review', 'cool_review', 'text', 'date', 
        'text_length', 'user_compliments_total', 'user_friends_count', 
        'user_is_elite', 'review_count' # Original user review count needed for Filter E
        # 'yelping_since' # Not strictly needed after date drop
    ]
    # Ensure all needed columns exist before selecting
    existing_cols_to_keep = [col for col in cols_to_keep_for_phase1 if col in chunk_merged.columns]
    processed_chunks.append(chunk_merged[existing_cols_to_keep])
    print(f"  Chunk {i+1} processing complete. Appended {len(chunk_merged)} reviews.")

    # 5. Memory Cleanup
    del chunk_merged
    gc.collect()

print(f"\nFinished processing all chunks. Total reviews processed: {total_reviews_processed}")
print(f"Total reviews kept after chunk-level processing: {total_reviews_kept_chunk_phase}")

Starting review processing in chunks of 500000...



--- Processing Chunk 1 --- (500000 reviews)
  Merging with users...
  Reviews after merge: 500000 (Removed 0 due to missing users)
  Performing feature engineering...
    Dropped 0 rows due to NaT dates.
  Applying review-level filters...
    Filter A (Text Length >= 50): Kept 499398 reviews
    Filter F (Low Usefulness): Kept 455494 reviews


C:\Users\wiztu\AppData\Local\Temp\ipykernel_13968\1114296421.py:62: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  contains_url_mask = chunk_merged['text'].str.contains(url_pattern, regex=True, na=False)


    Filter 2A (URLs/Emails): Kept 454744 reviews
  Total reviews removed in this chunk by review-level filters: 45256
  Chunk 1 processing complete. Appended 454744 reviews.

--- Processing Chunk 2 --- (500000 reviews)
  Merging with users...
  Reviews after merge: 499996 (Removed 4 due to missing users)
  Performing feature engineering...
    Dropped 0 rows due to NaT dates.
  Applying review-level filters...
    Filter A (Text Length >= 50): Kept 499469 reviews
    Filter F (Low Usefulness): Kept 457006 reviews


C:\Users\wiztu\AppData\Local\Temp\ipykernel_13968\1114296421.py:62: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  contains_url_mask = chunk_merged['text'].str.contains(url_pattern, regex=True, na=False)


...

    Filter 2A (URLs/Emails): Kept 455486 reviews
  Total reviews removed in this chunk by review-level filters: 44513
  Chunk 12 processing complete. Appended 455486 reviews.

--- Processing Chunk 13 --- (500000 reviews)
  Merging with users...
  Reviews after merge: 499995 (Removed 5 due to missing users)
  Performing feature engineering...
    Dropped 0 rows due to NaT dates.
  Applying review-level filters...
    Filter A (Text Length >= 50): Kept 499424 reviews
    Filter F (Low Usefulness): Kept 456983 reviews


C:\Users\wiztu\AppData\Local\Temp\ipykernel_13968\1114296421.py:62: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  contains_url_mask = chunk_merged['text'].str.contains(url_pattern, regex=True, na=False)


    Filter 2A (URLs/Emails): Kept 456227 reviews
  Total reviews removed in this chunk by review-level filters: 43768
  Chunk 13 processing complete. Appended 456227 reviews.

--- Processing Chunk 14 --- (490280 reviews)
  Merging with users...
  Reviews after merge: 490275 (Removed 5 due to missing users)
  Performing feature engineering...
    Dropped 0 rows due to NaT dates.
  Applying review-level filters...
    Filter A (Text Length >= 50): Kept 489805 reviews
    Filter F (Low Usefulness): Kept 448357 reviews


C:\Users\wiztu\AppData\Local\Temp\ipykernel_13968\1114296421.py:62: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  contains_url_mask = chunk_merged['text'].str.contains(url_pattern, regex=True, na=False)


    Filter 2A (URLs/Emails): Kept 447620 reviews
  Total reviews removed in this chunk by review-level filters: 42655
  Chunk 14 processing complete. Appended 447620 reviews.

Finished processing all chunks. Total reviews processed: 6990280
Total reviews kept after chunk-level processing: 6372756


## Combine Processed Chunks

In [6]:
print("\nCombining processed chunks...")
if processed_chunks:
    reviews_combined_df = pd.concat(processed_chunks, ignore_index=True)
    print(f"Combined DataFrame shape: {reviews_combined_df.shape}")
    # Clear chunk list from memory
    del processed_chunks
    gc.collect()
else:
    print("No chunks were processed or kept. Exiting.")
    # Handle exit or create an empty DataFrame if needed
    reviews_combined_df = pd.DataFrame() # Or raise an error

# Ensure date is datetime type after concat
if not reviews_combined_df.empty:
    reviews_combined_df['date'] = pd.to_datetime(reviews_combined_df['date'], errors='coerce')


Combining processed chunks...
Combined DataFrame shape: (6372756, 14)


## Phase 1: Rule-Based Filtering (User Aggregation)

In [7]:
print("--- Baseline Metrics (After Chunking/Combining) ---")

# scalar baselines you’ll need later
baseline_review_count   = 0
baseline_user_count     = 0
baseline_avg_text_len   = np.nan
baseline_avg_useful     = np.nan
baseline_user_agg       = pd.Series(dtype=float)   # user-level means

# -------- sanity-check --------
if "reviews_combined_df" not in locals():
    print("ERROR: reviews_combined_df not found.")
    reviews_combined_df = pd.DataFrame()

# -------- metrics --------
if not reviews_combined_df.empty:
    baseline_review_count = len(reviews_combined_df)
    baseline_user_count   = reviews_combined_df["user_id"].nunique()
    print(f"Combined Unique User Count: {baseline_user_count}")
    print(f"Combined Review Count: {baseline_review_count}")

    # quick check for required columns
    req_cols = [
        "stars", "text_length", "useful_review",
        "user_compliments_total", "user_friends_count", "user_is_elite"
    ]
    missing_cols = [c for c in req_cols if c not in reviews_combined_df.columns]
    if missing_cols:
        print(f"WARNING: Missing cols for baseline metrics: {missing_cols}")
    else:
        print("Star-Rating Distribution (%):")
        print(reviews_combined_df["stars"]
              .value_counts(normalize=True).sort_index()*100)

        # ---------- store scalar review-level baselines ----------
        baseline_avg_text_len = reviews_combined_df["text_length"].mean()
        baseline_avg_useful   = reviews_combined_df["useful_review"].mean()

        print(f"Combined Avg Text Length: {baseline_avg_text_len:.2f}")
        print(f"Combined Avg Useful Votes / Review: {baseline_avg_useful:.2f}")

        # ---------- user-level means ----------
        df_for_agg = reviews_combined_df.drop_duplicates("user_id").copy()
        baseline_user_agg = (
            df_for_agg[["user_compliments_total",
                        "user_friends_count",
                        "user_is_elite"]]
            .rename(columns={
                "user_compliments_total": "avg_compliments",
                "user_friends_count"   : "avg_friends",
                "user_is_elite"        : "pct_elite"
            })
            .mean()
        )

        print("\nBaseline Avg User Metrics (Unique Users):")
        print(f"  Avg Compliments: {baseline_user_agg['avg_compliments']:.2f}")
        print(f"  Avg Friends    : {baseline_user_agg['avg_friends']:.2f}")
        print(f"  % Elite Users  : {baseline_user_agg['pct_elite']*100:.2f}%")

    # hand off for Stage-1 filters (no .copy() to save RAM)
    reviews_stage1_df = reviews_combined_df

else:
    print("reviews_combined_df is empty – nothing to summarise")
    reviews_stage1_df = pd.DataFrame()


--- Baseline Metrics (After Chunking/Combining) ---
Combined Unique User Count: 1836520
Combined Review Count: 6372756
Star-Rating Distribution (%):
stars
1    15.981986
2     8.171818
3    10.227977
4    20.934867
5    44.683352
Name: proportion, dtype: float64
Combined Avg Text Length: 609.63
Combined Avg Useful Votes / Review: 1.29

Baseline Avg User Metrics (Unique Users):
  Avg Compliments: 15.81
  Avg Friends    : 53.67
  % Elite Users  : 4.92%


### Apply User-Level Filters (Iteratively)

In [8]:
# Filters B, C, D, E are applied here on the combined data
if not reviews_stage1_df.empty:
    # (B) Filter by Review Bursts
    print("\n--- Filter B: Review Bursts ---")
    count_before = len(reviews_stage1_df)
    users_before = reviews_stage1_df['user_id'].nunique()

    # Calculate reviews per user per day
    reviews_per_day = reviews_stage1_df.groupby([reviews_stage1_df['user_id'], reviews_stage1_df['date'].dt.date])\
                                         .size().reset_index(name='reviews_per_day')

    # Find max reviews per day for each user
    max_reviews_per_user = reviews_per_day.groupby('user_id')['reviews_per_day'].max().reset_index(name='max_reviews_per_day')

    # Merge max back to the main df
    reviews_stage1_df = pd.merge(reviews_stage1_df, max_reviews_per_user, on='user_id', how='left')

    avg_max_burst_before = reviews_stage1_df['max_reviews_per_day'].mean()

    # Apply filter
    MAX_BURST_RATE = 10 # Reviews per day
    users_to_remove_burst = reviews_stage1_df[reviews_stage1_df['max_reviews_per_day'] > MAX_BURST_RATE]['user_id'].unique()
    reviews_stage1_df = reviews_stage1_df[~reviews_stage1_df['user_id'].isin(users_to_remove_burst)]

    count_after = len(reviews_stage1_df)
    users_after = reviews_stage1_df['user_id'].nunique()
    # Recalculate mean only if df is not empty
    avg_max_burst_after = reviews_stage1_df['max_reviews_per_day'].mean() if not reviews_stage1_df.empty else 0

    print(f"Users identified with burst rate > {MAX_BURST_RATE}: {len(users_to_remove_burst)}")
    print(f"Reviews removed: {count_before - count_after}")
    print(f"Users removed: {users_before - users_after}")
    print(f"Avg max reviews/day before: {avg_max_burst_before:.2f}, after: {avg_max_burst_after:.2f}")

    # Drop the temporary column
    reviews_stage1_df.drop(columns=['max_reviews_per_day'], inplace=True)
    gc.collect()
else:
    print("Skipping Filter B: Input DataFrame is empty.")


--- Filter B: Review Bursts ---
Users identified with burst rate > 10: 4329
Reviews removed: 359159
Users removed: 4329
Avg max reviews/day before: 3.60, after: 2.81


In [9]:
print("--- Baseline Metrics (After Chunking/Combining) ---")
baseline_review_count = baseline_user_count = 0
baseline_user_agg = pd.Series(dtype=float)          # will become a Series later

# ---------- sanity-check that the combined reviews exist ----------
if "reviews_combined_df" not in locals():
    print("ERROR: reviews_combined_df not found.  Did the previous cell run?")
    reviews_combined_df = pd.DataFrame()            # placeholder so the rest doesn’t crash

# ---------- basic counts ----------
if not reviews_combined_df.empty:
    baseline_review_count = len(reviews_combined_df)
    if "user_id" in reviews_combined_df.columns:
        baseline_user_count = reviews_combined_df["user_id"].nunique()
        print(f"Combined Unique User Count: {baseline_user_count}")
    else:
        print("WARNING: 'user_id' missing – cannot count users")

    print(f"Combined Review Count: {baseline_review_count}")

    # ---------- quick univariate stats ----------
    required_cols_baseline = [
        "stars", "text_length", "useful_review", "user_compliments_total",
        "user_friends_count", "user_is_elite"
    ]
    missing_cols = [c for c in required_cols_baseline if c not in reviews_combined_df.columns]

    if missing_cols:
        print(f"\nWARNING: Missing columns needed for baseline metrics: {missing_cols}")
    else:
        print("Combined Star Rating Distribution (%):")
        print(reviews_combined_df["stars"].value_counts(normalize=True).sort_index()*100)
        print(f"Combined Avg Text Length: {reviews_combined_df['text_length'].mean():.2f}")
        print(f"Combined Avg Useful Votes / Review: {reviews_combined_df['useful_review'].mean():.2f}")

        # ---------- aggregate per-user metrics ----------
        df_for_agg = reviews_combined_df.drop_duplicates("user_id").copy()
        numeric_fix = {
            "user_compliments_total": "avg_compliments",
            "user_friends_count": "avg_friends",
            "user_is_elite": "pct_elite",
        }

        # make sure all three cols are numeric / boolean
        for col in numeric_fix:
            if not pd.api.types.is_numeric_dtype(df_for_agg[col]):
                df_for_agg[col] = pd.to_numeric(df_for_agg[col], errors="coerce").fillna(0)

        # mean() returns a Series we can rename
        baseline_user_agg = (
            df_for_agg[list(numeric_fix)]
            .mean()
            .rename(numeric_fix)
        )

        # ---------- pretty-print ----------
        print("\nBaseline Avg User Metrics (Unique Users):")
        print(f"  Avg Compliments: {baseline_user_agg['avg_compliments']:.2f}")
        print(f"  Avg Friends Count: {baseline_user_agg['avg_friends']:.2f}")
        print(f"  % Elite Users: {baseline_user_agg['pct_elite']*100:.2f}%")

    # ---------- keep copy for Stage 1 filters ----------
    reviews_stage1_df = reviews_combined_df

    # free memory once baseline metrics are safely stored
    if not missing_cols and not baseline_user_agg.empty:
        del reviews_combined_df
        gc.collect()
        print("\n(Freed memory from reviews_combined_df)")
else:
    print("reviews_combined_df is empty – nothing to summarise")
    reviews_stage1_df = pd.DataFrame()

--- Baseline Metrics (After Chunking/Combining) ---
Combined Unique User Count: 1836520
Combined Review Count: 6372756
Combined Star Rating Distribution (%):
stars
1    15.981986
2     8.171818
3    10.227977
4    20.934867
5    44.683352
Name: proportion, dtype: float64
Combined Avg Text Length: 609.63
Combined Avg Useful Votes / Review: 1.29

Baseline Avg User Metrics (Unique Users):
  Avg Compliments: 15.81
  Avg Friends Count: 53.67
  % Elite Users: 4.92%

(Freed memory from reviews_combined_df)


In [10]:
if not reviews_stage1_df.empty:
    # (D) Filter by Rating Deviation
    print("\n--- Filter D: Rating Deviation ---")
    count_before = len(reviews_stage1_df)
    users_before = reviews_stage1_df['user_id'].nunique()

    # Calculate average star rating per business (using current filtered reviews)
    business_avg_stars = reviews_stage1_df.groupby('business_id')['stars'].mean().reset_index(name='business_avg_rating')

    # Merge business average rating
    reviews_stage1_df = pd.merge(reviews_stage1_df, business_avg_stars, on='business_id', how='left')

    # Calculate deviation for each review
    reviews_stage1_df['rating_deviation'] = (reviews_stage1_df['stars'] - reviews_stage1_df['business_avg_rating']).abs()

    # Calculate average deviation per user
    user_avg_deviation = reviews_stage1_df.groupby('user_id')['rating_deviation'].mean().reset_index(name='user_avg_deviation')

    # Merge user average deviation
    reviews_stage1_df = pd.merge(reviews_stage1_df, user_avg_deviation, on='user_id', how='left')

    avg_dev_before = reviews_stage1_df['user_avg_deviation'].mean()

    # Apply filter (excluding Elite users)
    MAX_AVG_DEVIATION = 1.8
    condition = (
        (reviews_stage1_df['user_avg_deviation'] > MAX_AVG_DEVIATION) & 
        (reviews_stage1_df['user_is_elite'] == False)
    )
    users_to_remove_deviation = reviews_stage1_df[condition]['user_id'].unique()
    reviews_stage1_df = reviews_stage1_df[~reviews_stage1_df['user_id'].isin(users_to_remove_deviation)]

    count_after = len(reviews_stage1_df)
    users_after = reviews_stage1_df['user_id'].nunique()
    avg_dev_after = reviews_stage1_df['user_avg_deviation'].mean() if not reviews_stage1_df.empty else 0

    print(f"Users identified with avg deviation > {MAX_AVG_DEVIATION} (non-elite): {len(users_to_remove_deviation)}")
    print(f"Reviews removed: {count_before - count_after}")
    print(f"Users removed: {users_before - users_after}")
    print(f"Avg user rating deviation before: {avg_dev_before:.2f}, after: {avg_dev_after:.2f}")

    # Drop temporary columns
    reviews_stage1_df.drop(columns=['business_avg_rating', 'rating_deviation', 'user_avg_deviation'], inplace=True)
    gc.collect()
else:
    print("Skipping Filter D: Input DataFrame is empty.")


--- Filter D: Rating Deviation ---
Users identified with avg deviation > 1.8 (non-elite): 324410
Reviews removed: 491430
Users removed: 324410
Avg user rating deviation before: 1.03, after: 0.92


In [11]:
if not reviews_stage1_df.empty:
    # (E) Filter by Low Engagement (Combined Rule)
    print("\n--- Filter E: Low Engagement ---")

    count_before  = len(reviews_stage1_df)
    users_before  = reviews_stage1_df['user_id'].nunique()

    # ------------------------------------------------------------------
    # 1. make sure the user-level review_count column exists
    # ------------------------------------------------------------------
    if 'review_count' not in reviews_stage1_df.columns:
        reviews_stage1_df = reviews_stage1_df.merge(
            users_df[['user_id', 'review_count']],
            on='user_id', how='left', suffixes=('', '_user')
        )

    # ------------------------------------------------------------------
    # 2. build mask & list of users to drop
    # ------------------------------------------------------------------
    low_engagement_mask = (
        (reviews_stage1_df['user_compliments_total'] == 0) &
        (reviews_stage1_df['user_friends_count']    == 0) &
        (~reviews_stage1_df['user_is_elite']) &                       # faster than == False
        (reviews_stage1_df['review_count'].fillna(0) <= 5)
    )
    users_to_remove = reviews_stage1_df.loc[low_engagement_mask, 'user_id'].unique()

    # ------------------------------------------------------------------
    # 3. quick stats – guard against “no rows” first
    # ------------------------------------------------------------------
    if users_to_remove.size:
        removed_stats = (
            reviews_stage1_df
            .loc[reviews_stage1_df['user_id'].isin(users_to_remove),
                 ['user_compliments_total', 'user_friends_count']]
            .mean()
            .rename({'user_compliments_total': 'avg_compliments',
                     'user_friends_count'    : 'avg_friends'})
        )
    else:  # create a Series of NaNs so .loc below never errors
        removed_stats = pd.Series({'avg_compliments': np.nan, 'avg_friends': np.nan})

    kept_stats = (
        reviews_stage1_df
        .loc[~reviews_stage1_df['user_id'].isin(users_to_remove),
             ['user_compliments_total', 'user_friends_count']]
        .mean()
        .rename({'user_compliments_total': 'avg_compliments',
                 'user_friends_count'    : 'avg_friends'})
    )

    # ------------------------------------------------------------------
    # 4. apply the filter
    # ------------------------------------------------------------------
    reviews_stage1_df = reviews_stage1_df[~reviews_stage1_df['user_id'].isin(users_to_remove)]

    count_after = len(reviews_stage1_df)
    users_after = reviews_stage1_df['user_id'].nunique()

    # ------------------------------------------------------------------
    # 5. reporting
    # ------------------------------------------------------------------
    print(f"Users identified as low engagement: {users_to_remove.size}")
    print(f"  Removed Avg Compliments: {removed_stats['avg_compliments']:.2f}, "
          f"Avg Friends: {removed_stats['avg_friends']:.2f}")
    print(f"  Kept Avg Compliments: {kept_stats['avg_compliments']:.2f}, "
          f"Avg Friends: {kept_stats['avg_friends']:.2f}")
    print(f"Reviews removed: {count_before - count_after}")
    print(f"Users   removed: {users_before  - users_after}")

    gc.collect()
else:
    print("Skipping Filter E: Input DataFrame is empty.")



--- Filter E: Low Engagement ---
Users identified as low engagement: 369006
  Removed Avg Compliments: 0.00, Avg Friends: 0.00
  Kept Avg Compliments: 212.50, Avg Friends: 147.50
Reviews removed: 503899
Users   removed: 369006


### Stage 1 Evaluation & Saving

In [12]:
# ------------------------------------------------------------
# Stage-1 Final Metrics
# ------------------------------------------------------------
print("\n--- Stage 1 Final Metrics ---")

if not reviews_stage1_df.empty:
    final_review_count_s1 = len(reviews_stage1_df)
    final_user_count_s1   = reviews_stage1_df["user_id"].nunique()

    print(f"Final Review Count (Stage 1): {final_review_count_s1}")
    print(f"Final Unique User Count (Stage 1): {final_user_count_s1}")

    print("Star-Rating Distribution (% - Stage 1):")
    print(reviews_stage1_df["stars"]
          .value_counts(normalize=True).sort_index()*100)

    print(f"Final Avg Text Length            : "
          f"{reviews_stage1_df['text_length'].mean():.2f}")
    print(f"Final Avg Useful Votes per Review: "
          f"{reviews_stage1_df['useful_review'].mean():.2f}")

    # ---------- per-user metrics ----------
    final_user_agg_s1 = (
        reviews_stage1_df.drop_duplicates("user_id")
        .agg({
            "user_compliments_total": "mean",
            "user_friends_count"   : "mean",
            "text_length"          : "mean",
            "useful_review"        : "mean",
            "user_is_elite"        : "mean"
        })
        .rename({
            "user_compliments_total": "avg_compliments",
            "user_friends_count"   : "avg_friends",
            "text_length"          : "avg_text_length",
            "useful_review"        : "avg_useful_votes",
            "user_is_elite"        : "pct_elite"
        })
    )

    print("\nFinal Avg User Metrics (Unique Users - Stage 1):")
    print(f"  Avg Compliments: {final_user_agg_s1['avg_compliments']:.2f}")
    print(f"  Avg Friends    : {final_user_agg_s1['avg_friends']:.2f}")
    print(f"  % Elite Users  : {final_user_agg_s1['pct_elite']*100:.2f}%")

    # ---------- baseline vs Stage-1 comparison ----------
    print("\n--- Comparison: Baseline vs Stage 1 ---")
    print(f"Reviews : {baseline_review_count} ➔ {final_review_count_s1} "
          f"({final_review_count_s1 / baseline_review_count * 100:.1f}%)")
    print(f"Users   : {baseline_user_count} ➔ {final_user_count_s1} "
          f"({final_user_count_s1 / baseline_user_count * 100:.1f}%)")
    print(f"Avg Text Length  : {baseline_avg_text_len:.2f} "
          f"➔ {final_user_agg_s1['avg_text_length']:.2f}")
    print(f"Avg Useful Votes : {baseline_avg_useful:.2f} "
          f"➔ {final_user_agg_s1['avg_useful_votes']:.2f}")
    print(f"Avg Compliments  : {baseline_user_agg['avg_compliments']:.2f} "
          f"➔ {final_user_agg_s1['avg_compliments']:.2f}")
    print(f"Avg Friends      : {baseline_user_agg['avg_friends']:.2f} "
          f"➔ {final_user_agg_s1['avg_friends']:.2f}")
    print(f"% Elite Users    : {baseline_user_agg['pct_elite']*100:.2f}% "
          f"➔ {final_user_agg_s1['pct_elite']*100:.2f}%")

else:
    print("Stage 1 DataFrame is empty. No metrics to calculate.")
    final_review_count_s1 = final_user_count_s1 = 0



--- Stage 1 Final Metrics ---
Final Review Count (Stage 1): 5377427
Final Unique User Count (Stage 1): 1143104
Star-Rating Distribution (% - Stage 1):
stars
1    10.879199
2     7.975301
3    11.481458
4    23.774790
5    45.889251
Name: proportion, dtype: float64
Final Avg Text Length            : 614.66
Final Avg Useful Votes per Review: 1.38

Final Avg User Metrics (Unique Users - Stage 1):
  Avg Compliments: 25.19
  Avg Friends    : 74.83
  % Elite Users  : 7.91%

--- Comparison: Baseline vs Stage 1 ---
Reviews : 6372756 ➔ 5377427 (84.4%)
Users   : 1836520 ➔ 1143104 (62.2%)
Avg Text Length  : 609.63 ➔ 522.83
Avg Useful Votes : 1.29 ➔ 0.88
Avg Compliments  : 15.81 ➔ 25.19
Avg Friends      : 53.67 ➔ 74.83
% Elite Users    : 4.92% ➔ 7.91%


In [13]:
# Qualitative Check (Stage 1)
print("\n--- Qualitative Check (Stage 1) ---")
if not reviews_stage1_df.empty:
    kept_reviews_s1 = reviews_stage1_df
    # To find removed reviews, we need to compare against the state *before* Stage 1 filters
    # This requires reloading the combined data or keeping a copy, which might use too much memory.
    # Simplified check: Show samples of kept reviews.
    print(f"Total kept after Stage 1: {len(kept_reviews_s1)}")

    if len(kept_reviews_s1) >= 10:
        print("\nSample Kept Reviews (Stage 1):")
        print(kept_reviews_s1.sample(min(10, len(kept_reviews_s1)))[['review_id', 'user_id', 'stars', 'text']])
    else:
        print("\nNot enough kept reviews to sample.")
    # Note: Comparing removed reviews is harder with chunking without reloading.
else:
    print("Stage 1 DataFrame is empty. No qualitative check possible.")


--- Qualitative Check (Stage 1) ---
Total kept after Stage 1: 5377427

Sample Kept Reviews (Stage 1):
                      review_id                 user_id  stars  \
5308559  2AZFGhEn4dXscJeJPR1IDg  UsQgl-9GE70Dga7Dvu6DUg      5   
3888991  K1lqJvBUiSibv15PeeoVFQ  lDfLMSRQmuG4pihc456pbQ      4   
3684099  AuY3dtRDkyWGhJAzDPHHaw  qQ4v3-TykvfHF7YmpfUifg      4   
3238320  p7UH-OFyvufxbtfgTaJ0CA  iJE3RnVcfHM3jsM4I6WFCw      3   
2635374  MTPF000pWZmNxEZFxAinPA  3ToghYK-MNVVx43wC-iGhA      1   
3154023  YwG8JPZ9LjdUnNLR0Yb7rw  t6QnunqfeTDNFPG311DFoA      3   
2592239  aYJrtHBRjVpVHL11P_saEw  b5IQGA8fXby6CLjeEELUgQ      5   
5555453  0xPdrxVXVqEeJO2AacppeA  m8q0X8bt7UAlawSjbG0c7Q      4   
6022162  rDy1enDTqAXMKqrK0z0eUg  UZ8_xqhiguIYb9Lu2Wu8og      3   
3659898  DVXaagL-hKPDYMQT0xQZ6w  22Y8hc4NYWPa2D6ffodddg      2   

                                                      text  
5308559  This was our first time here and we wanted to ...  
3888991  i've been to fork for lunch and dinner.

In [14]:
# Save Cleaned Data (Stage 1)
print("\n--- Saving Stage 1 Data ---")
if not reviews_stage1_df.empty:
    # Select relevant columns for reviews output
    review_cols_to_keep_output = ['review_id', 'user_id', 'business_id', 'stars', 'useful_review', 'funny_review', 'cool_review', 'text', 'date']
    reviews_stage1_cleaned_df = reviews_stage1_df[review_cols_to_keep_output]

    output_reviews_s1_path = os.path.join(output_dir, 'reviews_stage1_cleaned.json')
    reviews_stage1_cleaned_df.to_json(output_reviews_s1_path, orient='records', lines=True, date_format='iso')
    print(f"Saved Stage 1 cleaned reviews ({len(reviews_stage1_cleaned_df)}) to: {output_reviews_s1_path}")

    # Filter original users_df to keep only those remaining
    remaining_user_ids_s1 = reviews_stage1_cleaned_df['user_id'].unique()
    users_stage1_cleaned_df = users_df[users_df['user_id'].isin(remaining_user_ids_s1)].copy() # Use copy to avoid SettingWithCopyWarning

    output_users_s1_path = os.path.join(output_dir, 'users_stage1_cleaned.json')
    users_stage1_cleaned_df.to_json(output_users_s1_path, orient='records', lines=True, date_format='iso')
    print(f"Saved Stage 1 cleaned users ({len(users_stage1_cleaned_df)}) to: {output_users_s1_path}")

    # Prepare for Stage 2 - Keep the filtered reviews DataFrame
    reviews_stage2_df = reviews_stage1_df.copy()
    del reviews_stage1_df # Free memory from the copy used for saving
    del reviews_stage1_cleaned_df
    gc.collect()
else:
    print("Stage 1 DataFrame is empty. Cannot save.")
    reviews_stage2_df = pd.DataFrame() # Ensure it exists for next phase check
    users_stage1_cleaned_df = pd.DataFrame() # Ensure it exists


--- Saving Stage 1 Data ---
Saved Stage 1 cleaned reviews (5377427) to: ../yelp_dataset/cleaned_data\reviews_stage1_cleaned.json
Saved Stage 1 cleaned users (1143104) to: ../yelp_dataset/cleaned_data\users_stage1_cleaned.json


## Phase 2: Basic Text & Content Heuristics

### Apply Text Filters

In [15]:
# Filter 2A (URLs/Emails) was moved to the chunk processing loop.
print("Filter 2A (URLs/Emails) was applied during chunk processing.")

Filter 2A (URLs/Emails) was applied during chunk processing.


In [ ]:
# (B) Filter by Repetition (Optional - requires NLTK)
print("\n--- Filter 2B: Text Repetition (Unique Ratio) ---")
nltk.download('punkt') # Ensure punkt tokenizer is downloaded
nltk.download('punkt_tab')
SKIP_REPETITION_FILTER = True # Set to True to skip this filter

if SKIP_REPETITION_FILTER:
    print("Skipping Filter 2B (Repetition) as configured.")
elif not reviews_stage2_df.empty:
    try:
        import nltk
        from nltk.corpus import stopwords
        from nltk.tokenize import word_tokenize

        try:
            nltk.data.find('corpora/stopwords')
        except nltk.downloader.DownloadError:
            print("Downloading NLTK stopwords...")
            nltk.download('stopwords')
        try:
            nltk.data.find('tokenizers/punkt')
        except nltk.downloader.DownloadError:
            print("Downloading NLTK punkt tokenizer...")
            nltk.download('punkt')

        stop_words = set(stopwords.words('english'))

        def calculate_unique_ratio(text):
            if pd.isna(text) or not isinstance(text, str) or len(text.strip()) == 0:
                return 1.0 # Assign high ratio if no text
            # Tokenize, lowercase, remove punctuation (simple regex), remove stopwords
            text = text.lower()
            text = re.sub(r'[^\w\s]', '', text) # Keep words and spaces
            tokens = word_tokenize(text)
            words = [word for word in tokens if word.isalpha() and word not in stop_words]
            if not words: # Handle case of no words after filtering
                return 1.0
            unique_words = set(words)
            ratio = len(unique_words) / len(words)
            return ratio

        print("Calculating unique ratio (may take time)...")
        reviews_stage2_df['unique_ratio'] = reviews_stage2_df['text'].apply(calculate_unique_ratio)

        avg_ratio_before = reviews_stage2_df['unique_ratio'].mean()

        MIN_UNIQUE_RATIO = 0.4
        count_before = len(reviews_stage2_df)
        users_before = reviews_stage2_df['user_id'].nunique()

        reviews_stage2_df = reviews_stage2_df[reviews_stage2_df['unique_ratio'] >= MIN_UNIQUE_RATIO]

        count_after = len(reviews_stage2_df)
        users_after = reviews_stage2_df['user_id'].nunique()
        avg_ratio_after = reviews_stage2_df['unique_ratio'].mean() if not reviews_stage2_df.empty else 0

        print(f"Reviews removed with unique ratio < {MIN_UNIQUE_RATIO}: {count_before - count_after}")
        print(f"Users removed: {users_before - users_after}")
        print(f"Avg unique ratio before: {avg_ratio_before:.3f}, after: {avg_ratio_after:.3f}")

        # Drop temporary column
        reviews_stage2_df.drop(columns=['unique_ratio'], inplace=True)
        gc.collect()

    except ImportError:
        print("Skipping Filter 2B (Repetition) because NLTK is not installed.")
        print("Please install NLTK (`pip install nltk`) and run the cell again if needed.")
else:
     print("Skipping Filter 2B: Input DataFrame is empty.")


--- Filter 2B: Text Repetition (Unique Ratio) ---


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wiztu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\wiztu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


Calculating unique ratio (may take time)...
Reviews removed with unique ratio < 0.4: 136
Users removed: 25
Avg unique ratio before: 0.896, after: 0.896


C:\Users\wiztu\AppData\Local\Temp\ipykernel_13968\1677047546.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_stage2_df.drop(columns=['unique_ratio'], inplace=True)


### Stage 2 Evaluation & Saving

In [19]:
# Calculate final metrics after Stage 2 filters
print("\n--- Stage 2 Final Metrics ---")
if not reviews_stage2_df.empty:
    final_review_count_s2 = len(reviews_stage2_df)
    final_user_ids_s2 = reviews_stage2_df['user_id'].unique()
    final_user_count_s2 = len(final_user_ids_s2)

    print(f"Final Review Count (Stage 2): {final_review_count_s2}")
    print(f"Final Unique User Count (Stage 2): {final_user_count_s2}")

    print("\n--- Comparison: Stage 1 vs Stage 2 ---")
    print(f"Reviews: {final_review_count_s1} -> {final_review_count_s2} - Removed in Stage 2: {final_review_count_s1 - final_review_count_s2}")
    print(f"Users: {final_user_count_s1} -> {final_user_count_s2} - Removed in Stage 2: {final_user_count_s1 - final_user_count_s2}")
else:
    print("Stage 2 DataFrame is empty. No metrics to calculate.")
    final_review_count_s2 = 0
    final_user_count_s2 = 0


--- Stage 2 Final Metrics ---
Final Review Count (Stage 2): 5377291
Final Unique User Count (Stage 2): 1143079

--- Comparison: Stage 1 vs Stage 2 ---
Reviews: 5377427 -> 5377291 - Removed in Stage 2: 136
Users: 1143104 -> 1143079 - Removed in Stage 2: 25


In [20]:
# Qualitative Check (Stage 2)
print("\n--- Qualitative Check (Stage 2) ---")
if not reviews_stage2_df.empty:
    kept_reviews_s2 = reviews_stage2_df
    # To find removed reviews, compare Stage 1 saved file vs current df
    try:
        temp_reviews_s1_df = pd.read_json(os.path.join(output_dir, 'reviews_stage1_cleaned.json'), lines=True)
        removed_review_ids_s2 = temp_reviews_s1_df[~temp_reviews_s1_df['review_id'].isin(kept_reviews_s2['review_id'])]['review_id']
        removed_reviews_s2 = temp_reviews_s1_df[temp_reviews_s1_df['review_id'].isin(removed_review_ids_s2)]
        print(f"Total kept: {len(kept_reviews_s2)}, Total removed in S2: {len(removed_reviews_s2)}")
        if len(removed_reviews_s2) >= 10:
            print("\nSample Removed Reviews (by Stage 2 Filters):")
            print(removed_reviews_s2.sample(min(10, len(removed_reviews_s2)))[['review_id', 'user_id', 'stars', 'text']])
        else:
            print("\nNot enough reviews removed in Stage 2 to sample.")
        del temp_reviews_s1_df, removed_reviews_s2 # cleanup
        gc.collect()
    except FileNotFoundError:
        print("Could not perform removed qualitative check: Stage 1 file not found.")
    except Exception as e:
         print(f"Error during Stage 2 qualitative check: {e}")
else:
    print("Stage 2 DataFrame is empty. No qualitative check possible.")


--- Qualitative Check (Stage 2) ---
Total kept: 5377291, Total removed in S2: 136

Sample Removed Reviews (by Stage 2 Filters):
                      review_id                 user_id  stars  \
309188   bSYueJW2QfnbfCbnwtGj6Q  zq-_GEZbFLKsDuvlJ1t2kA      5   
524267   MKNDWij0YIqpqP8l-iGT0A  bzREcvyIB1n1sozIHbrdNw      1   
2668224  95FSLmXd2KjeRXVLxTfFNQ  _1XrHcVtxeVWQHYVfKHKIQ      5   
19237    Qx1q2UxbsDgGzMo7dpNy0w  Jl4kC3eiGlEgEylXpOj7-Q      3   
1096800  GJiRo6Fn7korFtdNVrYb1g  cBQYOYNa6OtJxN5mZOqDkQ      5   
1454270  E9758dNn_831GI8PuXC_IA  pZQDbsJy9eJG8RyXwFcGNQ      5   
2305602  c3RU9GItoQR70T3hgUAGQw  gLzco4fFq3IFHn562zcKDA      4   
3003841  ZFys_wvnRPaDg3Q9oyNsvA  k0WffTIS1SJx64srck0VsQ      5   
775502   4NGiQ6fj6Mr7KQ9OW-NOXQ  XOjDtIL_Fs5b0Lm0486wTg      1   
2884672  2wf5pwxNcR1UW-S5S6X78Q  kQblyilW-V3k208BJg7mpQ      1   

                                                      text  
309188   Quick service.  Good food for the price.  Quai...  
524267   If I could gi

In [21]:
# Save Cleaned Data (Stage 2)
print("\n--- Saving Stage 2 Data ---")
if not reviews_stage2_df.empty:
    # Select columns for output
    reviews_stage2_cleaned_df = reviews_stage2_df[review_cols_to_keep_output]

    output_reviews_s2_path = os.path.join(output_dir, 'reviews_stage2_cleaned.json')
    reviews_stage2_cleaned_df.to_json(output_reviews_s2_path, orient='records', lines=True, date_format='iso')
    print(f"Saved Stage 2 cleaned reviews ({len(reviews_stage2_cleaned_df)}) to: {output_reviews_s2_path}")

    # Filter Stage 1 users based on remaining user_ids in Stage 2 reviews
    remaining_user_ids_s2 = reviews_stage2_cleaned_df['user_id'].unique()
    # Use the users_stage1_cleaned_df saved previously
    if not users_stage1_cleaned_df.empty:
        users_stage2_cleaned_df = users_stage1_cleaned_df[users_stage1_cleaned_df['user_id'].isin(remaining_user_ids_s2)].copy()
        output_users_s2_path = os.path.join(output_dir, 'users_stage2_cleaned.json')
        users_stage2_cleaned_df.to_json(output_users_s2_path, orient='records', lines=True, date_format='iso')
        print(f"Saved Stage 2 cleaned users ({len(users_stage2_cleaned_df)}) to: {output_users_s2_path}")
    else:
        print("Warning: Stage 1 users DataFrame was empty, cannot save Stage 2 users.")
        users_stage2_cleaned_df = pd.DataFrame() # Ensure exists

    # Clear Stage 2 DataFrames from memory
    del reviews_stage2_df
    del reviews_stage2_cleaned_df
    del users_stage1_cleaned_df # No longer needed
    gc.collect()
else:
    print("Stage 2 DataFrame is empty. Cannot save.")
    users_stage2_cleaned_df = pd.DataFrame() # Ensure exists for next phase


--- Saving Stage 2 Data ---
Saved Stage 2 cleaned reviews (5377291) to: ../yelp_dataset/cleaned_data\reviews_stage2_cleaned.json
Saved Stage 2 cleaned users (1143079) to: ../yelp_dataset/cleaned_data\users_stage2_cleaned.json


## Phase 3: Advanced Filtering (Isolation Forest)

In [ ]:
# Load Stage 2 Data
PHASE_3 = False
if PHASE_3==False:
    print("Skipping Phase 3")
else:
    print("--- Loading Stage 2 Data for Phase 3 ---")
    reviews_stage3_input_df = pd.DataFrame()
    users_stage3_input_df = pd.DataFrame()
    try:
        reviews_stage3_input_path = os.path.join(output_dir, 'reviews_stage1_cleaned.json')
        reviews_stage3_input_df = pd.read_json(reviews_stage3_input_path, lines=True)
        reviews_stage3_input_df['date'] = pd.to_datetime(reviews_stage3_input_df['date'], errors='coerce') # Ensure date is datetime
        print(f"Loaded Stage 2 reviews: {reviews_stage3_input_df.shape}")
    except FileNotFoundError:
        print(f"Error: Stage 2 review file not found at {reviews_stage3_input_path}")
    except Exception as e:
        print(f"Error loading Stage 2 reviews: {e}")

    try:
        users_stage3_input_path = os.path.join(output_dir, 'users_stage1_cleaned.json')
        users_stage3_input_df = pd.read_json(users_stage3_input_path, lines=True)
        print(f"Loaded Stage 2 users: {users_stage3_input_df.shape}")
    except FileNotFoundError:
        print(f"Error: Stage 2 user file not found at {users_stage3_input_path}")
    except Exception as e:
        print(f"Error loading Stage 2 users: {e}")

--- Loading Stage 2 Data for Phase 3 ---
Loaded Stage 2 reviews: (5377427, 9)
Loaded Stage 2 users: (1143104, 22)


In [ ]:
# Prepare Features for Isolation Forest
if PHASE_3:
    print("\nPreparing features for Isolation Forest...")
    features_df = pd.DataFrame()
    reviews_stage3_merged_df = pd.DataFrame()

    if not reviews_stage3_input_df.empty and not users_stage3_input_df.empty:
        # Merge user data back into reviews
        reviews_stage3_merged_df = pd.merge(reviews_stage3_input_df, users_stage3_input_df, on='user_id', how='inner', suffixes=('_review', '_user'))
        print(f"Merged Stage 2 reviews and users: {reviews_stage3_merged_df.shape}")

        # Recalculate user_avg_deviation based on current (Stage 2) reviews
        print("  Recalculating business/user average ratings...")
        # 1. Calculate business avg stars from Stage 2 reviews
        business_avg_stars_s2 = reviews_stage3_merged_df.groupby('business_id')['stars'].mean().reset_index(name='business_avg_rating_s2')
        # 2. Merge into the merged df
        reviews_stage3_merged_df = pd.merge(reviews_stage3_merged_df, business_avg_stars_s2, on='business_id', how='left')
        # 3. Calculate review deviation
        reviews_stage3_merged_df['rating_deviation_s2'] = (reviews_stage3_merged_df['stars'] - reviews_stage3_merged_df['business_avg_rating_s2']).abs()
        # 4. Calculate user average deviation
        user_avg_deviation_s2 = reviews_stage3_merged_df.groupby('user_id')['rating_deviation_s2'].mean().reset_index(name='user_avg_deviation_s2')
        # 5. Merge user avg deviation back
        reviews_stage3_merged_df = pd.merge(reviews_stage3_merged_df, user_avg_deviation_s2, on='user_id', how='left')

        # Add other features from user data
        print("  Adding other features...")
        reviews_stage3_merged_df['text_length'] = reviews_stage3_merged_df['text'].astype(str).str.len()
        compliment_cols_s3 = [col for col in reviews_stage3_merged_df.columns if col.startswith('compliment_') and col != 'compliment_count'] # Exclude tip compliment if present
        reviews_stage3_merged_df['user_compliments_total'] = reviews_stage3_merged_df[compliment_cols_s3].sum(axis=1)
        reviews_stage3_merged_df['user_friends_count'] = reviews_stage3_merged_df['friends'].apply(count_friends) # Use function defined earlier

        # Select features for the model
        feature_cols = [
            'text_length',
            'useful_review',
            'stars',
            'review_count', # User's total review count (from users_df)
            'user_compliments_total',
            'user_friends_count',
            'user_avg_deviation_s2',
            'average_stars' # User's average rating (from users_df)
        ]

        # Ensure all feature columns exist
        missing_cols = [col for col in feature_cols if col not in reviews_stage3_merged_df.columns]
        if missing_cols:
            print(f"Warning: Missing required feature columns: {missing_cols}. Isolation Forest cannot run.")
        else:
            features_df = reviews_stage3_merged_df[feature_cols].copy()

            # Handle NaNs (e.g., fill with median or mean)
            print("  Handling NaNs in features...")
            for col in feature_cols:
                if features_df[col].isnull().any():
                    median_val = features_df[col].median()
                    features_df[col].fillna(median_val, inplace=True)
                    print(f"    Filled NaNs in '{col}' with median ({median_val:.2f})")

            print("Feature preparation complete.")
    else:
        print("Skipping feature preparation: Input DataFrames from Stage 2 are empty or failed to load.")


Preparing features for Isolation Forest...
Merged Stage 2 reviews and users: (5377427, 30)
  Recalculating business/user average ratings...
  Adding other features...
  Handling NaNs in features...
Feature preparation complete.


In [ ]:
if PHASE_3:
    # Train & Predict with Isolation Forest
    reviews_stage3_filtered_df = pd.DataFrame() # Initialize

    if not features_df.empty and not reviews_stage3_merged_df.empty:
        print("\nTraining Isolation Forest and predicting anomalies...")
        try:
            # Initialize model
            # contamination='auto' lets the algorithm estimate the proportion of outliers
            # Or set explicitly, e.g., contamination=0.01 for 1%
            iso_forest = IsolationForest(contamination='auto', random_state=42, n_jobs=-1) # Use all available cores

            # Fit and predict
            # fit_predict returns 1 for inliers, -1 for outliers
            reviews_stage3_merged_df['anomaly_score'] = iso_forest.fit_predict(features_df)

            n_outliers = (reviews_stage3_merged_df['anomaly_score'] == -1).sum()
            print(f"Identified {n_outliers} potential outliers (anomaly_score = -1).")

            # Filter out anomalies
            count_before = len(reviews_stage3_merged_df)
            users_before = reviews_stage3_merged_df['user_id'].nunique()

            reviews_stage3_filtered_df = reviews_stage3_merged_df[reviews_stage3_merged_df['anomaly_score'] == 1].copy()

            count_after = len(reviews_stage3_filtered_df)
            users_after = reviews_stage3_filtered_df['user_id'].nunique()

            print(f"Reviews removed by Isolation Forest: {count_before - count_after}")
            print(f"Users removed: {users_before - users_after}")

            # Optional: Examine features of outliers vs inliers
            outliers_df = reviews_stage3_merged_df[reviews_stage3_merged_df['anomaly_score'] == -1]
            if not outliers_df.empty and not reviews_stage3_filtered_df.empty:
                print("\nFeature Averages (Outliers vs Inliers):")
                print(pd.DataFrame({'Outliers': outliers_df[feature_cols].mean(), 'Inliers': reviews_stage3_filtered_df[feature_cols].mean()}))
            del outliers_df
            gc.collect()

        except Exception as e:
            print(f"Error during Isolation Forest execution: {e}")
            # Keep all reviews if IF fails
            reviews_stage3_filtered_df = reviews_stage3_merged_df.copy()
            if 'anomaly_score' in reviews_stage3_filtered_df.columns:
                reviews_stage3_filtered_df.drop(columns=['anomaly_score'], inplace=True)
    else:
        print("Skipping Isolation Forest: No features prepared or input data empty.")
        # If IF was skipped, the 'filtered' df is the 'merged' df from before
        if not reviews_stage3_merged_df.empty:
            reviews_stage3_filtered_df = reviews_stage3_merged_df.copy()
        else:
            reviews_stage3_filtered_df = pd.DataFrame() # Ensure it's an empty df


Training Isolation Forest and predicting anomalies...


C:\Users\wiztu\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\wiztu\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\wiztu\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\wiztu\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in 

Identified 449400 potential outliers (anomaly_score = -1).
Reviews removed by Isolation Forest: 449400
Users removed: 70117

Feature Averages (Outliers vs Inliers):
                           Outliers     Inliers
text_length             1200.358104  561.243182
useful_review              5.967488    0.958524
stars                      2.863992    3.948860
review_count             714.281157  102.802536
user_compliments_total  2103.808358   40.022286
user_friends_count       795.661907   88.396274
user_avg_deviation_s2      0.924540    0.874446
average_stars              3.317155    3.889810


### Stage 3 Evaluation & Saving (Final)

In [14]:
# Final Evaluation after Stage 3
print("\n--- Stage 3 Final Metrics (Overall Final) ---")
if not reviews_stage3_filtered_df.empty:
    final_review_count_s3 = len(reviews_stage3_filtered_df)
    final_user_ids_s3 = reviews_stage3_filtered_df['user_id'].unique()
    final_user_count_s3 = len(final_user_ids_s3)
    final_business_ids_s3 = reviews_stage3_filtered_df['business_id'].unique()
    final_business_count_s3 = len(final_business_ids_s3)

    print(f"Final Review Count (End of Stage 3): {final_review_count_s3}")
    print(f"Final Unique User Count (End of Stage 3): {final_user_count_s3}")
    print(f"Final Unique Business Count (End of Stage 3): {final_business_count_s3}")

    print("\n--- Comparison: Stage 2 vs Stage 3 ---")
    print(f"Reviews: {final_review_count_s2} -> {final_review_count_s3} - Removed in Stage 3: {final_review_count_s2 - final_review_count_s3}")
    print(f"Users: {final_user_count_s2} -> {final_user_count_s3} - Removed in Stage 3: {final_user_count_s2 - final_user_count_s3}")
else:
    print("Stage 3 DataFrame is empty. No final metrics.")
    final_review_count_s3 = 0
    final_user_count_s3 = 0
    final_business_count_s3 = 0


--- Stage 3 Final Metrics (Overall Final) ---
Final Review Count (End of Stage 3): 4928027
Final Unique User Count (End of Stage 3): 1072987
Final Unique Business Count (End of Stage 3): 149570

--- Comparison: Stage 2 vs Stage 3 ---


NameError: name 'final_review_count_s2' is not defined

In [ ]:
# Save Final Cleaned Data
print("\n--- Saving Final Cleaned Data ---")

if not reviews_stage3_filtered_df.empty:
    # Final Reviews
    reviews_final_cleaned_df = reviews_stage3_filtered_df[review_cols_to_keep_output].copy() # Select original review columns
    output_reviews_final_path = os.path.join(output_dir, 'reviews_final_cleaned.json')
    reviews_final_cleaned_df.to_json(output_reviews_final_path, orient='records', lines=True, date_format='iso')
    print(f"Saved final cleaned reviews ({len(reviews_final_cleaned_df)}) to: {output_reviews_final_path}")

    # Final Users (Filter the users loaded from Stage 2)
    if not users_stage3_input_df.empty:
        users_final_cleaned_df = users_stage3_input_df[users_stage3_input_df['user_id'].isin(final_user_ids_s3)].copy()
        output_users_final_path = os.path.join(output_dir, 'users_final_cleaned.json')
        users_final_cleaned_df.to_json(output_users_final_path, orient='records', lines=True, date_format='iso')
        print(f"Saved final cleaned users ({len(users_final_cleaned_df)}) to: {output_users_final_path}")
    else:
        print("Warning: Stage 2 users input was empty, cannot save final users.")
        users_final_cleaned_df = pd.DataFrame()

    # Final Businesses (Filter the original business_df)
    if not business_df.empty:
        business_final_cleaned_df = business_df[business_df['business_id'].isin(final_business_ids_s3)].copy()
        output_business_final_path = os.path.join(output_dir, 'business_final_cleaned.json')
        business_final_cleaned_df.to_json(output_business_final_path, orient='records', lines=True, date_format='iso')
        print(f"Saved final cleaned businesses ({len(business_final_cleaned_df)}) to: {output_business_final_path}")
    else:
        print("Warning: Original business_df was empty, cannot save final businesses.")
        business_final_cleaned_df = pd.DataFrame()

    # Final Memory Cleanup
    del reviews_stage3_input_df
    del users_stage3_input_df
    del reviews_stage3_merged_df
    del reviews_stage3_filtered_df
    del features_df
    del reviews_final_cleaned_df
    del users_final_cleaned_df
    del business_final_cleaned_df
    del users_df # Original users no longer needed
    del business_df # Original business no longer needed
    gc.collect()
else:
    print("Stage 3 DataFrame is empty. Cannot save final files.")


--- Saving Final Cleaned Data ---
Stage 3 DataFrame is empty. Cannot save final files.


In [ ]:
# Business burst goes here



3-day burst percentiles:
 90%    4.0
95%    5.0
98%    6.0
99%    7.0
Name: count, dtype: float64

5-day burst percentiles:
 90%     6.0
95%     7.0
98%     9.0
99%    10.0
Name: count, dtype: float64

7-day burst percentiles:
 90%     9.0
95%     9.0
98%    11.0
99%    13.0
Name: count, dtype: float64

→ Using K_BURST=3, N_BURST=7


KeyError: 'review_count'

## Phase 4: Post-Cleaning Summary

**Summary of Cleaning Process (Chunking Refactor):**

*   **Initial Data:** Started with Y users, Z businesses (Review count processed in chunks).
*   **Chunk Processing:** Reviews loaded in chunks. Merged with users, feature engineered, and filtered by text length (A), low usefulness (F), and URLs/Emails (2A) within each chunk.
*   **Combined Data (Baseline):** `baseline_review_count` reviews and `baseline_user_count` users remaining after combining chunks.
*   **After Stage 1 (User Aggregation Rules):** Reduced to `final_review_count_s1` reviews and `final_user_count_s1` users.
    *   Removed users/reviews based on burst rate (B), short activity windows (C), high rating deviation (D, non-elite), and low engagement (E).
*   **After Stage 2 (Text Heuristics):** Reduced to `final_review_count_s2` reviews and `final_user_count_s2` users.
    *   Removed reviews based on text repetition (B, optional/if NLTK present).
*   **After Stage 3 (Isolation Forest):** Reduced to `final_review_count_s3` reviews and `final_user_count_s3` users.
    *   Removed reviews identified as anomalies based on selected features.

**Final Output:**

*   `reviews_final_cleaned.json`: Contains `final_review_count_s3` cleaned reviews.
*   `users_final_cleaned.json`: Contains `final_user_count_s3` users corresponding to the final reviews.
*   `business_final_cleaned.json`: Contains `final_business_count_s3` businesses corresponding to the final reviews.

These files are located in the `yelp_dataset/cleaned_data/` directory.

In [ ]:
print("\n--- Cleaning Process Complete ---")
print(f"Final review count: {final_review_count_s3 if 'final_review_count_s3' in locals() else 'N/A'}")
print(f"Final user count: {final_user_count_s3 if 'final_user_count_s3' in locals() else 'N/A'}")
print(f"Final business count: {final_business_count_s3 if 'final_business_count_s3' in locals() else 'N/A'}")
print(f"Cleaned files saved in: {output_dir}")


--- Cleaning Process Complete ---
Final review count: 0
Final user count: 0
Final business count: 0


NameError: name 'output_dir' is not defined